### Importing libraries

In [68]:
import pandas as pd
import numpy as np
import time 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

### Getting target list 

In [69]:
datatracks = pd.read_csv('../data/clean/api_tracks1_clean.csv', encoding='latin1')

In [70]:
datatracks.head(2)

,track_id,track_name,album_id,artist_id,artist_name
0,4ZY1RPoLfezEqu03BI2axd,Sgt. Pepper's Lonely Hearts Club Band,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta
1,0QLyr8xasb6tgGkdPWYw8d,With a Little Help From My Friends,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta


In [71]:
track_ids = datatracks['track_id'].unique().tolist()

In [72]:
track_ids[0:2]

['4ZY1RPoLfezEqu03BI2axd', '0QLyr8xasb6tgGkdPWYw8d']

### Spotify Connection 

In [73]:
#%pip install pyarrow
#%pip install joblib
#%pip install tqdm
#%pip install spotipy
#%pip install python-dotenv
#%pip install requests 

In [74]:
import requests as rq
import pyarrow
from joblib import Parallel, delayed
from tqdm import tqdm
import pymongo
from passwords import *

In [75]:
# Obtener el TOKEN

AUTH_URL = "https://accounts.spotify.com/api/token"

creds = {
    "grant_type": "client_credentials",
    "client_id": Client_ID,
    "client_secret": Client_secret
}

response = rq.post(AUTH_URL, 
                         data=creds, 
                         headers={"Content-Type": "application/x-www-form-urlencoded"})


TOKEN = response.json()['access_token']

# Conexión con la API

url = "https://api.spotify.com/"

headers = {"Authorization": f'Bearer {TOKEN}'}

response = rq.get(url,headers=headers)
response

<Response [200]>

### Getting Tracks Information 

In [76]:
track_ids = datatracks['track_id'].unique().tolist()

In [77]:
track_ids[0:2]

['4ZY1RPoLfezEqu03BI2axd', '0QLyr8xasb6tgGkdPWYw8d']

In [78]:
#endpoint
#https://api.spotify.com/v1/audio-features/{id}

In [79]:
#test 
url = 'https://api.spotify.com/v1/audio-features/0QLyr8xasb6tgGkdPWYw8d'

In [80]:
res = rq.get(url, headers = headers)
res

<Response [429]>

In [66]:
resjson = res.json()
type(resjson)

dict

In [67]:
resjson.keys()

dict_keys(['error'])

In [64]:
resjson['danceability']

KeyError: 'danceability'

In [ ]:
# Construct a list of URLs for querying track audio features
track_urls = [f'https://api.spotify.com/v1/audio-features/{track_id}' for track_id in track_ids]

# Create a list of dictionaries with track information and corresponding feature URLs
track_url_list = [{'track_id': track_id, 'url': url} for track_id, url in zip(track_ids, track_urls)]

In [ ]:
# List to store the retrieved track data
track_data_list = []

# Function to search and retrieve track information
def search_track_info(track_url):
    time.sleep(1)  # Pause between queries to avoid getting banned

    try:
        response = rq.get(track_url['url'], headers=headers)
        response.raise_for_status()  # Check for request errors
        data = response.json()

        # Extract necessary data from the response
        track_data_list.append({
            'track_id': track_url['track_id'],
            'danceability': data.get('danceability'),
            'popularity': data.get('popularity'),
            'energy': data.get('energy'),
            'acousticness': data.get('acousticness'),
            'valence': data.get('valence'),
            'instrumentalness': data.get('instrumentalness'),
            'tempo': data.get('tempo'),
            'loudness': data.get('loudness')
        })

    except Exception as e:
        print(f"An error occurred: {e}")
        track_data_list.append({
            'track_id': None,
            'danceability': None,
            'popularity': None,
            'energy': None,
            'acousticness': None,
            'valence': None,
            'instrumentalness': None,
            'tempo': None,
            'loudness': None
        })

# Execute the function for each track URL
for track_url in track_url_list:
    search_track_info(track_url)

# Create a DataFrame from the list of dictionaries
data_tracks2 = pd.DataFrame(track_data_list)

In [ ]:
data_tracks2.head()

In [ ]:
data_tracks.shape

In [ ]:
data_tracks.to_csv('../data/clean/api_tracks1_clean.csv', index=False)